## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [168]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    real: float
    dual: float = 0.0

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual():
                return Dual(self.real + other.real, self.dual + other.dual)
            case Number():
                return Dual(float(other) + self.real, self.dual)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual():
                return Dual(self.real * other.real, self.real * other.dual + self.dual * other.real)
            case Number():
                return Dual(float(other) * self.real, float(other) * self.dual)  
    
    def __neg__(self) -> "Dual":
        return Dual(-self.real, -self.dual)
    
    def __pos__(self) -> "Dual":
        return Dual(+self.real, +self.dual)
    
    def __abs__(self) -> "Dual":
        return Dual(abs(self.real), abs(self.dual))

    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual():
                return Dual(self.real - other.real, self.dual - other.dual)
            case Number():
                return Dual(float(other) - self.real, self.dual)
    
    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        if other.real == 0:
            raise ZeroDivisionError("Attepting to divide by zero")
        
        if isinstance(other, Dual):
            other = other
        else: 
            other = Dual(real=other)
        
        real = self.real / other.real
        dual = (self.dual * other.real - self.real * other.dual)/ (other.real * other.real)
        
        return Dual(real, dual)
    
    def __rtruediv__(self, other: float) -> "Dual":
        if self.real == 0:
            raise ZeroDivisionError("Attempting to divide by a zero")
            
        return Dual(real=other) / self

    def __pow__(self, power: Union["Dual", Number]) -> "Dual": 
        if isinstance(power, Dual):
            other = power
            real = self.real ** other.real
            dual = real * ((self.dual / self.real) * other.real + log(self.real) * other.dual)
            return Dual(real, dual) 
        
        real = self.real ** power
        dual = real * (self.dual / self.real) * power
        return Dual(real, dual)
    
    def __rpow__(self, other: Union["Dual", Number]) -> "Dual":        
        if other.real == 0:
            raise Exception("log(0) is not defined")

        return Dual(other) ** self

    __radd__ = __add__
    __rmul__ = __mul__  
    __rsub__ = __sub__
    

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).dual

In [169]:
import pytest
from math import cos, sin, exp, log

#Реализуем проверку каждого отдельного метода с использованием assert
# Проверка унарных операций __neg__, __pos__, __abs__. Без реализации этих методов в классе будет возникать TypeError: bad operand type for unary (+, -, abs()): 'Dual'
# __neg__:
f = lambda x: -x 
f_diff = diff(f)
assert f_diff(2) == -1, "__neg__ is not right"

# __pos__:
f = lambda x: +x 
f_diff = diff(f)
assert f_diff(2) == 1, "__pos__ is not right"

# __abs__:
f = lambda x: abs(-x)
f_diff = diff(f)
assert f_diff(2) == 1, "__abs__ is not right"

#Проверим поддержку операции вычитания __sub__:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x - 2 * x - 2

f_diff = diff(f)
assert f_diff(2) == 18, "__sub__ is not right"

#Проверим поддержку операции деления __truediv__:
#1. Проверка деления на ноль
def f(x: float) -> float:
    return x / 0

f_diff = diff(f)

def test_zero_division():
    with pytest.raises(ZeroDivisionError):
        f_diff(2)

#2.
def f(x: float) -> float:
    return 5 / (x * x) - x * x / 2 - 2

f_diff = diff(f)
assert f_diff(2) == -3.25, "__truediv__ is not right"

#Проверим операцию возведения в степень __pow__:
#1
f = lambda x: x ** 4 
f_diff = diff(f)
assert f_diff(2) == 32, "__pow__ is not right"

#2
f = lambda x: 4 ** x
f_diff = diff(f)
assert round(f_diff(2), 4) == 22.1807, "__pow__ is not right"

#3
f = lambda x: x ** x
f_diff = diff(f)
assert round(f_diff(2), 5) == 6.77259, "__pow__ is not right"

#4
f = lambda x: 0 ** x
f_diff = diff(f)
def test_negative_base():
    with pytest.raises(AssertionError):
        f_diff(2)

#5
f = lambda x: x ** 5
f_diff = diff(f)
def test_negative_base():
    with pytest.raises(ZeroDivisionError):
        f_diff(0)

#Проверим компиляцию всех операций:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x / (x ** 4 + 2) - 2 * x - 2

f_diff = diff(f)
f_diff(2)
assert round(f_diff(2), 4) == -2.8642, "wrong answer"

## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [170]:
def sin_d(dual_num) -> "Dual":
    real = sin(dual_num.real)
    dual = cos(dual_num.real) * dual_num.dual
    return Dual(real, dual)

def cos_d(dual_num) -> "Dual":
    real = cos(dual_num.real)
    dual = -sin(dual_num.real) * dual_num.dual
    return Dual(real, dual)

def exp_d(dual_num) -> "Dual":
    real = exp(dual_num.real)
    dual = real * dual_num.dual
    return Dual(real, dual)

def log_d(dual_num) -> "Dual":
    if dual_num.real == 0:
        raise AssertionError(f"log(0) is not defined")

    real = log(dual_num.real)
    dual = dual_num.dual / dual_num.real
    return Dual(real, dual)

In [171]:
from math import pi, exp

#Реализуем проверку каждого отдельного метода с использованием assert
def test_sin():
    f = lambda x: sin_d(x ** 2)
    f_diff = diff(f)
    assert f_diff(pi) == 2 * pi * cos(pi**2), "sin() is not right"

def test_cos():
    f = lambda x: cos_d(x ** 2)
    f_diff = diff(f)
    assert f_diff(pi) == -2 * pi * sin(pi**2), "cos() is not right"

def test_exp():
    f = lambda x: exp_d(x ** 2)
    f_diff = diff(f)
    assert f_diff(pi) == 2 * pi * exp(pow(pi, 2)), "exp() is not right"

def test_log():
    f = lambda x: log_d(x ** 2)
    f_diff = diff(f)
    assert f_diff(pi) == 2 / pi, "log() is not right"

    f = lambda x: log_d(x)
    f_diff =diff(f)
    with pytest.raises(AssertionError):
        f_diff(0)


In [172]:
test_sin()
test_cos()
test_exp()
test_log()

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [115]:
from scipy.misc import derivative

def f(x: float) -> float:
    return (x**3) - 2 * (x**2) * (x**2 + 2) + (x**3 + x + 5)

print(diff(f)(2))
print(derivative(f, 2.))
print('Right answer is -55')

-55.0
-69.0
Right answer is -55


In [116]:
def f(x: float) -> float:
    return 81 * x / (x + (x**2))

print(diff(f)(2))
print(derivative(f, 2.))

print('Right answer is -9')

-9.0
-10.125
Right answer is -9


In [117]:
def f(x: float) -> float:
    return sin(x)/(cos(x)+(x**2))

print(derivative(f, 2.))

def f(x: float) -> float:
    return sin_d(x)/(cos_d(x)+(x**2))

print(diff(f)(2))

print('Right answer is -0.334925')

-0.2643422639065857
-0.3349249363278422
Right answer is -0.334925


## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [372]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

from numpy import real

@dataclass
class Dual:
    real: float
    dual: dict

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual():
                real = self.real + other.real
                dual = {}
                for key in self.dual:
                    dual[key] = self.dual[key]
                for key in other.dual:
                    if key in dual:
                        dual[key] += other.dual[key]
                    else:
                        dual[key] = other.dual[key]
                return Dual(real, dual)
            case Number():
                return Dual(float(other) + self.real, self.dual)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual():
                real = self.real * other.real
                dual = {}
                for key in self.dual:
                    dual[key] = self.dual[key] * other.real
                for key in other.dual:
                    if key in dual:
                        dual[key] += other.dual[key] * self.real
                    else:
                        dual[key] = other.dual[key] * self.real
                return Dual(real, dual)
            case Number():
                dual = {}
                for key in self.dual:
                    dual[key] = self.dual[key] * other
                return Dual(self.real * other, dual)
    
    def __neg__(self) -> "Dual":
        dual = {}
        for key in self.dual:
            dual[key] = self.dual[key] * (-1)
        return Dual(-self.real, self.dual)
    
    def __pos__(self) -> "Dual":
        dual = {}
        for key in self.dual:
            dual[key] = self.dual[key] * (+1)
        return Dual(self.real, +self.dual)
    
    def __abs__(self) -> "Dual":
        dual = {}
        for key in self.dual:
            dual[key] = abs(self.dual[key])
        return Dual(abs(self.real), self.dual)

    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual():
                real = self.real - other.real
                dual = {}
                for key in self.dual:
                    dual[key] = self.dual[key]
                for key in other.dual:
                    if key in dual:
                        dual[key] -= other.dual[key]
                    else:
                        dual[key] = -other.dual[key]
                return Dual(real, dual)
            case Number():
                return Dual(self.real - float(other), self.dual)
    
    def div_neg(self, other):
        dual = {}
        for key in other.dual:
            dual[key] = other.dual[key]*(-1)
        return Dual(other.real,dual)

    def __truediv__(self,other):
        match other:
            case Dual():
                x = other.real
                new_arg = self.div_neg(other)
                num = Dual(self.real, self.dual)
                num_modified = num*new_arg
                dual = {}
                for key in num_modified.dual:
                    dual[key] = num_modified.dual[key] / (x*x)
                return Dual(num_modified.real / (x*x), dual)
            case Number():
                dual = {}
                for key in self.dual:
                    dual[key] = self.dual[key] / other
                return Dual(self.real / other, dual)

    def __rtruediv__(self,other):
        x = self.real
        den = Dual(self.real, self.dual)
        new_arg = self.div_neg(den)
        num_modified = other*new_arg
        dual = {}
        for key in num_modified.dual:
            dual[key] = num_modified.dual[key] / (x*x)
        return Dual(num_modified.real / (x*x), dual)

    def __pow__(self, power: Union["Dual", Number]) -> "Dual": 
        real = self.real
        dual = {}
        for key in self.dual:
            dual[key] = power * self.dual[key] * (real ** (power - 1))

        return Dual(real ** power, dual)
    
    def __rpow__(self, other: Union["Dual", Number]) -> "Dual": 
        if other.real == 0:
            raise Exception("log(0) is not defined")
        real = other ** self.real
        dual = {}
        for key in self.dual:
            dual[key] = (other ** self.real) * (self.dual[key] * log(other))
        
        return Dual(real, dual)

    __radd__ = __add__
    __rmul__ = __mul__

    def __str__(self):
        s = 'f = ' + str(round(self.real,6)) + '\n'
        for key in self.dual:
            s += 'f' + key + ' = ' + str(round(self.dual[key],6)) + '\n'
        return s

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x, y, z: func(Dual(x, {'x': 1}), Dual(y, {'y': 1}), Dual(z, {'z': 1})).dual


In [373]:
def sin_d(dual_num) -> "Dual":
    dual = {}
    real = sin(dual_num.real)
    for key in dual_num.dual:
        dual[key] = cos(dual_num.real) * dual_num.dual[key]
    return Dual(real, dual)

def cos_d(dual_num) -> "Dual":
    dual = {}
    real = cos(dual_num.real)
    for key in dual_num.dual:
        dual[key] = -sin(dual_num.real) * dual_num.dual[key] 
    return Dual(real, dual)

def exp_d(dual_num) -> "Dual":
    dual = {}
    real = exp(dual_num.real)
    for key in dual_num.dual:
        dual[key] = real * dual_num.dual[key]
    return Dual(real, dual)

def log_d(dual_num) -> "Dual":
    if dual_num.real == 0:
        raise AssertionError("log(0) is not defined")
    
    dual = {}
    real = log(dual_num.real)
    for key in dual_num.dual:
        dual[key] = dual_num.dual[key] / dual_num.real
    return Dual(real, dual)

In [374]:
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  

f_diff = diff(f)
f_diff(10, 10, 10)

{'x': 10, 'y': 5, 'z': 1}

Проверим на нескольких функциях:

$$f(x,y) = 6x^3 + 3x^2y^2 - 2y^3$$
$$f_x = \frac {\partial f} {\partial x} = 6x(3x + y^2)$$
$$f_y = \frac {\partial f} {\partial y} = 6y(x^2 -2)$$
В точке: x = 2, y = 3

$$f = 102$$
$$f_x = 180$$
$$f_y = 18$$

In [375]:
x = Dual(real=2, dual={'x': 1})
y = Dual(real=3, dual={'y': 1})

f = 6 * (x**3) + 3 * (x**2)*(y**2) - 2 * (y**3)
print(f)

f = 102
fx = 180
fy = 18



$$f(x,y) = \frac {36x} {x + y^2}$$
$$f_x = \frac {\partial f} {\partial x} = \frac{36y^2}{(x+y^2)^2}$$
$$f_y = \frac {\partial f} {\partial y} = -\frac{72xy}{(x+y^2)^2}$$

В точке: x = 2, y = 1

$$f = 24$$
$$f_x = 4$$
$$f_y = -16$$

In [376]:
def f(x: float, y: float, z: float) -> float:
    return 36*x / (x + y**2)

f_diff = diff(f)
f_diff(2, 1, 0)

{'x': 4.0, 'y': -16.0}

$$f(x,y) = \frac {\sin(y)} {cos(x) + x**2}$$
$$f_x = \frac {\partial f} {\partial x} = \frac{(\sin(x)-2x)\sin(y)}{(x^2+\cos(x))^2}$$
$$f_y = \frac {\partial f} {\partial y} = \frac{\cos(y)}{cos(x) + x^2}$$

В точке: $$x = \pi, y = \pi, z = 1$$

$$f = 0$$
$$f_x = 0$$
$$f_y = -0.112745$$

In [377]:
x = Dual(pi, {'x': 1})
y = Dual(pi, {'y': 1})

f = (sin_d(y) / (cos_d(x) + x**2))
print(f)

f = 0.0
fy = -0.112745
fx = -0.0

